# Proyecto Junio 2023: Ensamble de modelos predictivos

## Autores
- Juan Carlos López Veiga
- Miguel Manzano Álvarez


Para el desarollo de este proyecto se tendran en cuenta para la experimentación los ficheros _titanic.csv_ y _pcos.csv_ ubicados en la carpeta _datos_.

En primer lugar importaremos todo aquello necesario para el desarrollo del proyecto.

In [257]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from statistics import mode
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from math import sqrt


A continuación, gracias a la librería _pandas_, guardaremos en sus respectivas variables el conjunto de datos iniciales tanto de titanic como de pcos.

In [258]:
pcosData = pd.read_csv('./datos/pcos.csv', skiprows = 1, header = None,
                           names=['Age (yrs)', 'Weight (Kg)', 'Height(Cm)', 'BMI', 'Blood Group', 'Pulse rate(bpm)',
                                   'RR (breaths/min)', 'Hb(g/dl)', 'Cycle(R/I)', 'Cycle length(days)', 'Marriage Status (Yrs)',
                                     'Pregnant(Y/N)', 'No. of abortions', 'I beta-HCG(mIU/mL)', 'FSH(mIU/mL)', 'LH(mIU/mL)', 'FSH/LH', 'Hip(inch)',
                                       'Waist(inch)', 'Waist:Hip Ratio', 'TSH (mIU/L)', 'PRL(ng/mL)', 'Vit D3 (ng/mL)', 'PRG(ng/mL)', 'RBS(mg/dl)', 'Weight gain(Y/N)', 'hair growth(Y/N)',
                                         'Skin darkening (Y/N)', 'Hair loss(Y/N)', 'Pimples(Y/N)', 'Fast food (Y/N)', 'Reg.Exercise(Y/N)', 'BP _Systolic (mmHg)', 'BP _Diastolic (mmHg)', 'Follicle No. (L)',
                                           'Follicle No. (R)', 'Avg. F size (L) (mm)', 'Avg. F size (R) (mm)', 'Endometrium (mm)', 'PCOS (Y/N)'])


titanicData = pd.read_csv('./datos/titanic.csv', skiprows = 1, header = None,
                              names=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Initial', 'Age_band', 'Family_Size', 'Alone', 'Fare_cat', 'Deck', 'Title', 'Is_Married', 'Survived'])


La función _dataStandarization_ nos ayudará a estandarizar nuestros ficheros de entradas. Los argumentos requeridos por esta función son:
- data: Conjunto de datos a estandarizar.
- dataName: String que dará nombre a los ficheros resultantes. Por coherencia se recomienta que este sea 'titanic' o 'pcos' en función de los datos con los que se esté trabajando.
- excludedColumns: Columnas que se excluiran en el proceso de estandatización.

In [259]:

def dataStandarization (data, dataName, excludedColumns):
    if not isinstance (dataName, str):
        print("El argumento dataName debe ser una cadena de carateres.")
    
    elif not isinstance (excludedColumns, list):
        print("El argumento excludedColumns debe ser una lista con el nombre de las columnas a excluir.")

    else:
        
        data_copy = data.drop(excludedColumns, axis = 1)
        numerical_columns = [col for col in data.columns if col not in excludedColumns]
        columns_to_standarize = data[numerical_columns]

        scaler = StandardScaler()
        columns_to_standarize = scaler.fit_transform(columns_to_standarize)
        data[numerical_columns] = columns_to_standarize

        data.to_csv('./datos/' + dataName + '_standarized.csv', index = False)

        print("El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre " + dataName + "_standarized.csv")


Finalmente, definiremos aquellas columnas que excluiremos en el proceso de estandarización, en este caso, las variables objetivo y las variables _booleanas_, y procederemos a estandarizar los ficheros.

In [260]:
titanic_excluded = ['Sex', 'Alone', 'Is_Married', 'Survived']
pcos_excluded = ['Pregnant(Y/N)', 'Weight gain(Y/N)', 'hair growth(Y/N)', 'Skin darkening (Y/N)', 'Hair loss(Y/N)', 'Pimples(Y/N)', 'Fast food (Y/N)','Reg.Exercise(Y/N)', 'PCOS (Y/N)']

dataStandarization(titanicData, 'titanic', titanic_excluded)
dataStandarization(pcosData, 'pcos', pcos_excluded)

El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre titanic_standarized.csv
El nuevo fichero estandarizado se ha guardado en el directorio datos con el nombre pcos_standarized.csv


La función _dataSplit_ dividirá el fichero de entrada en un fichero de entrenamiento (2/3 del fichero original) y en un fichero de pruebas (1/3 del fichero original). Para ello requerirá los sigueintes argumentos:
- data: Conjunto de datos a estandarizar.
- dataName: String que dará nombre a los ficheros resultantes. Por coherencia se recomienta que este sea 'titanic' o 'pcos' en función de los datos con los que se esté trabajando.

In [261]:
def dataSplit (data, dataName):
    if not isinstance (dataName, str):
        print("El argumento dataName debe ser una cadena de carateres.")
    
    else:
        data_train, data_test = model_selection.train_test_split(data, test_size = 0.3, random_state = 99)
        data_train.to_csv('./datos/' + dataName + '_train.csv', index = False)
        data_test.to_csv('./datos/' + dataName + '_test.csv', index = False)

        print('Las dimensiones originales de los datos de entrada son: ', data.shape)
        print('El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre ' + dataName + '_train.csv, y sus dimensiones son: ', data_train.shape)
        print('El conjunto de pruebas se ha guardado en el directorio datos con el nombre ' + dataName + '_test.csv, y sus dimensiones son: ', data_test.shape)
        return data_train, data_test

Una vez definida dicha función, procedemos a crear nuestros nuevos ficheros.

In [262]:
titanicTrain, titanicTest = dataSplit(titanicData, 'titanic')
pcosTrain, pcosTest = dataSplit(pcosData, 'pcos')

Las dimensiones originales de los datos de entrada son:  (891, 16)
El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre titanic_train.csv, y sus dimensiones son:  (623, 16)
El conjunto de pruebas se ha guardado en el directorio datos con el nombre titanic_test.csv, y sus dimensiones son:  (268, 16)
Las dimensiones originales de los datos de entrada son:  (541, 40)
El conjunto de entrenamiento se ha guardado en el directorio datos con el nombre pcos_train.csv, y sus dimensiones son:  (378, 40)
El conjunto de pruebas se ha guardado en el directorio datos con el nombre pcos_test.csv, y sus dimensiones son:  (163, 40)


A estas alturas ya tenemos dos ficheros por cada conjunto inicial de datos, el de entrenamiento y el de pruebas. Sin embargo, procederemos a aplicar las técnicas _Bootstrapping_ y _Random Subspace Method_ al fichero de entrenamiento para así obtener nuevos conjuntos de datos con los que entrenar los diferentes modelos.

La función generadorConjuntosEntrenamiento necesitará los siguientes argumentos:
- fileName: String que dará nombre a los ficheros resultantes. Por coherencia se recomienta que este sea 'titanic' o 'pcos' en función de los datos con los que se esté trabajando.
- data: Conjunto de datos de entrenamiento.
- amountFiles: El número total de nuevos ficheros creados será igual a _amountFiles_ x _amountFiles_.

In [263]:

def generadorConjutosEntrenamiento(fileName, data, amountFiles):

    bFiles = sqrt(amountFiles)
    rsmFiles = sqrt(amountFiles)

    res= [] 

    col = data[data.columns[:-1]]
    
    objectiveVariable = data.loc[:, data.columns == data.columns[-1]]

    numCol=col.shape[1]

    for i in range (amountFiles):
        bootstrap_sample = data.sample(frac = 0.8, replace = True) 
        for j in range (amountFiles):    
            
            selcted_column = np.random.choice(numCol,size=int(np.sqrt(numCol)),replace = False)
            subspace_sample = col.iloc[:, selcted_column].copy() 
            subspace_sample[data.columns.values[-1]] = objectiveVariable

            res.append(subspace_sample)

            route = f'./datos/conjuntosEntrenamiento/{fileName}_trainSet_{i+1}.{j+1}.csv'
            subspace_sample.to_csv(route, index = False)
            print('Fichero creados en la ruta: ', route)

    return res

A continuación definiremos la función encargada del entrenamiento de modelos, la cual llamará a la función generadora definida previamente. Para ello definiremos previamente una función que nos será útil para dividir las variables objetivos del resto de variables.

También definiremos dos funciones, una que dada una lista de listas de valores, nos devuelva una lista con las modas de los valores de la misma posición, y otra que dado una lista de modelos entrenados y un conjuto de datos, nos prediga el valor de la variable objetivo utilizando la moda de los resultdos obtenidos en cada uno de los modelos.

A continuación definiremos las siguientes funciones:
- separarVariables: Dado un conjunto de datos, separa la variable objetivo del resto de variables.
- entrenamientoDeModelos: Algoritmo encargado de entrenar una serie de modelos a partir de un conjunto de entrenamiento, para ello necesitara diferentes argumentos:
    - data: Conjunto de datos de entrenamiento.
    - numModelos: Numero de modelos a entrenar, debido a als técnicas aplicadas, el resultado final de modelos es igual a numModelos * numModelos.
    - algoritmo: Argumento de tipo String, de valor "TREE" o "SGD" en función del algoritmo deseado para el entrenamiento de los modelos.
    - proporcionColumnas: Numero entre 0 y 1 que representa el porcentaje de columnas empleadas para el entrenamiento de los modelos.
    - fileName: String que representa el nombre del archivo, para seguir la coherencia del proyecto, esta será "titanic" o "pcos" en función del conjunto de datos.
- modasLista: Dada una lista de listas de numeros, devuelve una lista con las modas de los numeros de una misma posición.
- algoritmoPrediccion: A partir de unos datos de pruebas y una lista de modelos, predice el valor de la variable objetivo. Estas predicciones se basan en la moda del conjunto de modelos previamente entrenados.

In [264]:
def separarVariables(data):
    x = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    return x, y


def entrenamientoDeModelos(data, numModelos, algoritmo, proporcionColumnas, fileName):
    res = []

    if not 0 <= proporcionColumnas <= 1:
        print("El parametro proporcionColumnas debe ser un numero entre 0 y 1")

    else:

        if not isinstance(algoritmo, str): 
            print("El argumento algoritmo no es un String")
    
        else:

            training_data = generadorConjutosEntrenamiento(fileName, data, numModelos)

            for i in training_data:

                if algoritmo.upper() == 'TREE':
                    alg = DecisionTreeClassifier()
        
                elif  algoritmo.upper() == 'SGD':
                    alg = SGDClassifier()

                x, y = separarVariables(i)

                num_columns = int(proporcionColumnas * x.shape[1])
                selected_columns = np.random.choice(x.columns, size=num_columns, replace=False)
               
                selectedX = x[selected_columns]

                alg.fit(selectedX, y)

                res.append((alg,selectedX.columns))
    return res

def modasLista(list_of_lists):
    result = []
    list_length = len(list_of_lists[0])

    for i in range(list_length):
        elements = [lst[i] for lst in list_of_lists]    
        result.append(mode(elements))

    return result

def algoritmoPrediccion(datosTesteo, conjunto):
    ls = []
    for i in conjunto:
        pred = i[0].predict(datosTesteo[i[1]])
        ls.append(pred)
    return  modasLista(ls)

Una vez definidos nuestros algoritmos, procederemos a probar con unos ejemplos. Para ello llamaremos a las funciones definidas anteriormente, introduciendoles diferentes parámetros de entrada y obteniendo una lista de modelos para _titanic_ y otra para _pcos_ que nos ayudarán a hacer las predicciones correspondientes.

In [265]:
titanicModels = entrenamientoDeModelos(titanicTrain, 3, 'tree', 1, 'titanic')
pcosModels = entrenamientoDeModelos(pcosTrain, 4, 'tree', 1, 'pcos')

titanicPredictions = algoritmoPrediccion(titanicTest, titanicModels)
pcosPredictions = algoritmoPrediccion(pcosTest, pcosModels)

Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_1.1.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_1.2.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_1.3.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_2.1.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_2.2.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_2.3.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_3.1.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_3.2.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/titanic_trainSet_3.3.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_1.1.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_1.2.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainS

Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_1.4.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_2.1.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_2.2.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_2.3.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_2.4.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_3.1.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_3.2.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_3.3.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_3.4.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_4.1.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_4.2.csv
Fichero creados en la ruta:  ./datos/conjuntosEntrenamiento/pcos_trainSet_4.3.csv
Fichero creados 

Para evaluar el rendimiento de nuestros modelos nos fijaremos en dos métricas:
- Puntuación de precisión equilibrada: Promedio de la precisión de cada clase individual.
- F1: Media del equilibrio entre la precisión y el recall (tasa de verdaderos positivos y falsos negativos) del modelo.
Ambas métricas son proporcionadas por la librería _sklearn_, sin embargo, para facilitar su implementación se han definido sus respectivas funciones que requieren los siguientes argumentos de entrada:
- testingData: Conjunto de datos de pruebas (considerando unos datos de prueba de los que conocemos previamente la clasificación del mismo).
- predicted: Lista de valores que represente la calsificación del conjunto de pruebas realizada por los modelos entrenados.

En estas métricas, el valor óptimo sera 1 y el peor 0.

In [266]:

def balancedAccuracyScore(testingData, predicted):
    return balanced_accuracy_score(testingData.iloc[:,-1].tolist(), predicted)

def f1Score(testingData, predicted):
    return f1_score(testingData.iloc[:,-1].tolist(), predicted)


Finalmente observaremos los valores de las métricas del ejemplo de experimento realizado.

In [267]:
print("El valor de la métrica Puntuación de precisión equilibrada para el conjunto de pruebas de titanic es: ", balancedAccuracyScore(titanicTest, titanicPredictions))
print("El valor de la métrica Puntuación de precisión equilibrada para el conjunto de pruebas de pcos es: ", balancedAccuracyScore(pcosTest, pcosPredictions))

print("El valor de la métrica F1 para el conjunto de pruebas de titanic es: ", f1Score(titanicTest, titanicPredictions))
print("El valor de la métrica F1 para el conjunto de pruebas de pcos es: ", f1Score(pcosTest, pcosPredictions))

El valor de la métrica Puntuación de precisión equilibrada para el conjunto de pruebas de titanic es:  0.7501834189288334
El valor de la métrica Puntuación de precisión equilibrada para el conjunto de pruebas de pcos es:  0.7689649630343941
El valor de la métrica F1 para el conjunto de pruebas de titanic es:  0.6741573033707866
El valor de la métrica F1 para el conjunto de pruebas de pcos es:  0.7047619047619049
